# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

# 1. Tensors

---

## 1.1 Warm-up: numpy

### Fonctionnement global du code

**"Numpy"** est une bibliothèque qui prend en charge des tableaux et matrices, ainsi qu'une multitude de fonctions mathématiques opérant sur ces tableaux. Donc, elle n'est pas destinée pour implémenter des modèles d'apprentissage automatique. Toutefois, le code suivant est un exemple qui montre qu'on peut implémenter un modèle de **régression polynomiale** d’ordre 3 capable de prédire la valeur de la fonction **sinus**.
D'abord, on commence par définir les variables "x" et "y" (données d'entraînement) qui représentent l'entrée et la sortie cible du modèle, "a", "b", "c" et "d" qui sont les poids à apprendre par le modèle (initialisés aléatoirement) et "y_pred" qui est la prédiction du réseau. Le  processus d'apprentissage s'effectue par la minimisation d'une fonction d'erreur **"Distance euclidienne"** à travers un algorithme d'optimisation différentiable **"Descente de gradient"**.

### Principales variables

- **x** : ndarray à 1 dimension de taille 2000 d'éléments de type float64. Elle comprend 2000 valeurs aléatoires équidistantes comprises entre $- \pi$ et $\pi$ à utiliser pour l'apprentissage du modèle.
- **y** : ndarray à 1 dimension de taille 2000 d'éléments de type float64. Elle comprend les valeurs cibles $sin(x)$ à prédire par le modèle.
- **a** : float64. C'est le premier poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{0}$ du polynôme) et calculé lors de la forward pass.
- **b** : float64. C'est le deuxième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{1}$ du polynôme) et calculé lors de la forward pass.
- **c** : float64. C'est le troisième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{2}$ du polynôme) et calculé lors de la forward pass.
- **d** : float64. C'est le quatrième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{3}$ du polynôme) et calculé lors de la forward pass.
- **y_pred** : ndarray à 1 dimension de taille 2000 d'éléments de type float64. Elle comprend les prédictions de modèle. Son calcul s'effectue lors de la forward pass.
- **loss** : float64. C'est la fonction de perte du modèle "Distance euclidienne". Son calcul s'effectue lors de la forward pass.
- **grad_y_pred** : ndarray à 1 dimension de taille 2000 d'éléments de type float64. Elle comprend la dérivée de la fonction d'erreur par rapport à la variable de prédiction **y_pred**: $\frac {dL}{dy_{pred}}$. Son calcul s'effectue lors de la backward pass.
- **grad_a** : float64. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **a**: $\frac {dL}{da}$. Son calcul s'effectue lors de la backward pass.
- **grad_b** : float64. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **b**: $\frac {dL}{db}$. Son calcul s'effectue lors de la backward pass.
- **grad_c** : float64. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **c**: $\frac {dL}{dc}$. Son calcul s'effectue lors de la backward pass.
- **grad_d** : float64. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **d**: $\frac {dL}{dd}$. Son calcul s'effectue lors de la backward pass.
- **t** : int. C'est une variable comprise entre 0 et 1999, elle représente le compteur d'itérations dans la boucle d'apprentissage.
- **learning_rate** : float. C'est le taux d'apprentissage de la descente du gradient, fixé à 1e-6. Il contrôle combien les poids peuvent changer au cours de chaque itération.

### Code

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

## 1.2 PyTorch: Tensors

### Fonctionnement global du code

La bibliothèque **"Numpy"** est incapable d'effectuer des calculs sur des processeurs graphiques. Ce code nous introduit à un nouveau type d'objets: **les tenseurs**. Maintenant, on entraine comme l'étape précédente un modèle de **régression polynomiale** d’ordre 3 capable de prédire la valeur de la fonction **sinus** mais on remplace les **ndarray** de la bibliothèque **"Numpy"** par des **tensors** de la bibliothèque **"Pytorch"**. Cette dernière nous permet d'entraîner le modèle sur un **GPU**.

### Principales variables

- **dtype** : torch.dtype, fixé à torch.float. C'est un objet qui représente le type de données d'une torch.Tensor.
- **device** : torch.device. C'est un objet représentant l'appareil sur lequel une torch.Tensor est ou sera allouée (CPU ou GPU). selon ce code, les torch.Tensor peuvent être envoyés au CPU ou GPU (cuda:0), ce qui signifie "type='cuda', index=0".
- **x** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend 2000 valeurs aléatoires équidistantes comprises entre $- \pi$ et $\pi$ à utiliser pour l'apprentissage du modèle.
- **y** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend les valeurs cibles $sin(x)$ à prédire par le modèle.
- **a** : torch.FloatTensor, rang 0. C'est le premier poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{0}$ du polynôme) et calculé lors de la forward pass.
- **b** : torch.FloatTensor, rang 0. C'est le deuxième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{1}$ du polynôme) et calculé lors de la forward pass.
- **c** : torch.FloatTensor, rang 0. C'est le troisième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{2}$ du polynôme) et calculé lors de la forward pass.
- **d** : torch.FloatTensor, rang 0. C'est le quatrième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{3}$ du polynôme) et calculé lors de la forward pass.
- **y_pred** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend les prédictions de modèle. Son calcul s'effectue lors de la forward pass.
- **loss** : float. C'est la fonction de perte du modèle "Distance euclidienne". Son calcul s'effectue lors de la forward pass.
- **grad_y_pred** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend la dérivée de la fonction d'erreur par rapport à la variable de prédiction **y_pred**: $\frac {dL}{dy_{pred}}$. Son calcul s'effectue lors de la backward pass.
- **grad_a** : torch.FloatTensor, rang 0. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **a**: $\frac {dL}{da}$. Son calcul s'effectue lors de la backward pass.
- **grad_b** : torch.FloatTensor, rang 0. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **b**: $\frac {dL}{db}$. Son calcul s'effectue lors de la backward pass.
- **grad_c** : torch.FloatTensor, rang 0. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **c**: $\frac {dL}{dc}$. Son calcul s'effectue lors de la backward pass.
- **grad_d** : torch.FloatTensor, rang 0. C'est la dérivée partielle de la fonction d'erreur par rapport au coefficient **d**: $\frac {dL}{dd}$. Son calcul s'effectue lors de la backward pass.
- **t** : int. C'est une variable comprise entre 0 et 1999, elle représente le compteur d'itérations dans la boucle d'apprentissage.
- **learning_rate** : float. C'est le taux d'apprentissage de la descente du gradient, fixé à 1e-6. Il contrôle combien les poids peuvent changer au cours de chaque itération.

### Code

In [ ]:
# -*- coding: utf-8 -*-

import torch
import math


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

# 2. Autograd

---

## 2.1 PyTorch: Tensors and autograd

### Fonctionnement global du code

L'implémentation précédente présente une implémentation manuelle du forward et backward pass d'un réseau de neurones. En effet, durant le forward pass, on définit un graphe de calcul dans lequel les noeuds sont des tenseurs et les arêtes sont des fonctions. Ensuite, **Autograd** fournit une différenciation automatique qui automatise le calcul des gradients au cours du backward pass. 
Dans ce code, on entraine un modèle de **régression polynomiale** d’ordre 3 capable de prédire la valeur de la fonction **sinus** tout en profitant du package **Autograd**.

### Principales variables

- **dtype** : torch.dtype, fixé à torch.float. C'est un objet qui représente le type de données d'une torch.Tensor.
- **device** : torch.device. C'est un objet représentant l'appareil sur lequel une torch.Tensor est ou sera allouée (CPU ou GPU). selon ce code, les torch.Tensor peuvent être envoyés au CPU ou GPU (cuda:0), ce qui signifie "type='cuda', index=0".
- **x** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend 2000 valeurs aléatoires équidistantes comprises entre $- \pi$ et $\pi$ à utiliser pour l'apprentissage du modèle. Elle est définie lors du forward pass avec un paramètre **requires_grad=False** par défaut car on n'a pas besion de calculer son gradient lors du backward pass.
- **y** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend les valeurs cibles $sin(x)$ à prédire par le modèle. Elle est définie lors du forward pass avec un paramètre **requires_grad=False** par défaut car on n'a pas besion de calculer son gradient lors du backward pass.
- **a** : torch.FloatTensor, rang 0. C'est le premier poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{0}$ du polynôme) et calculé lors de la forward pass. Elle est définie lors du forward pass avec un paramètre **requires_grad=True** car on a besoin de calculer son gradient lors du backward pass.
- **b** : torch.FloatTensor, rang 0. C'est le deuxième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{1}$ du polynôme) et calculé lors de la forward pass. Elle est définie lors du forward pass avec un paramètre **requires_grad=True** car on a besoin de calculer son gradient lors du backward pass.
- **c** : torch.FloatTensor, rang 0. C'est le troisième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{2}$ du polynôme) et calculé lors de la forward pass. Elle est définie lors du forward pass avec un paramètre **requires_grad=True** car on a besoin de calculer son gradient lors du backward pass.
- **d** : torch.FloatTensor, rang 0. C'est le quatrième poids du modèle initialisé par une valeur aléatoire (c'est le coefficient de $x^{3}$ du polynôme) et calculé lors de la forward pass. Elle est définie lors du forward pass avec un paramètre **requires_grad=True** car on a besoin de calculer son gradient lors du backward pass.
- **y_pred** : torch.FloatTensor, rang 1, de taille 2000 d'éléments de type float32. Elle comprend les prédictions de modèle. Elle est définie lors du forward pass.
- **loss** : torch.FloatTensor, rang 0. C'est la fonction de perte du modèle "Distance euclidienne". Elle est définie lors du forward pass.
- **t** : int. C'est une variable comprise entre 0 et 1999, elle représente le compteur d'itérations dans la boucle d'apprentissage.
- **learning_rate** : float. C'est le taux d'apprentissage de la descente du gradient, fixé à 1e-6. Il contrôle combien les poids peuvent changer au cours de chaque itération.

### Code

In [ ]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')